In [ ]:
from __future__ import print_function

import sys
import numpy as np
import matplotlib.pyplot as plt 
import multiprocessing
import asyncio

from time import time
from pynq import Overlay
from pynq import allocate
from pynq import MMIO
from multiprocessing import Process, Manager, Value
from ctypes import c_char_p

sys.path.append('/home/xilinx')

ROM_SIZE = 0x2000 #8K

SOC_UP = 0x0000;
SOC_LA = 0x1000;
PL_AA_MB = 0x2000;
PL_AA = 0x2100;
SOC_IS = 0x3000;
SOC_AS = 0x4000;
SOC_CC = 0x5000;
PL_AS = 0x6000;
PL_IS = 0x7000;
PL_DMA = 0x8000;
FIR_DMA = 0x9000;

ol = Overlay("fsicIRQ.bit")
#ol.ip_dict
ipOUTPIN = ol.output_pin_0
ipPS = ol.caravel_ps_0
# ipReadROMCODE = ol.read_romcode_0

#Add for SPI
ip_QSPI = ol.axi_quad_spi_0

# ============================================
# AXI QuadSPI Control
# ============================================
XSP_DGIER_OFFSET = 0x1C
XSP_IISR_OFFSET = 0x20
XSP_IIER_OFFSET = 0x28
XSP_SRR_OFFSET = 0x40
XSP_CR_OFFSET = 0x60
XSP_SR_OFFSET = 0x64
XSP_DTR_OFFSET = 0x68
XSP_DRR_OFFSET = 0x6C
XSP_SSR_OFFSET = 0x70
XSP_TFO_OFFSET = 0x74
XSP_RFO_OFFSET = 0x78
XSP_REGISTERS = [0x40, 0x60, 0x64, 0x68, 0x6c, 0x70, 0x74, 0x78, 0x1c, 0x20, 0x28]

XSP_SRR_RESET_MASK = 0x0A
XSP_SR_TX_EMPTY_MASK = 0x04
XSP_SR_TX_FULL_MASK = 0x08
XSP_CR_TRANS_INHIBIT_MASK = 0x100
XSP_CR_LOOPBACK_MASK = 0x01
XSP_CR_ENABLE_MASK = 0x02
XSP_CR_MASTER_MODE_MASK = 0x04
XSP_CR_CLK_POLARITY_MASK = 0x08
XSP_CR_CLK_PHASE_MASK = 0x10
XSP_CR_TXFIFO_RESET_MASK = 0x20
XSP_CR_RXFIFO_RESET_MASK = 0x40
XSP_CR_MANUAL_SS_MASK = 0x80

SLAVE_NO_SELECTION = 0xFFFFFFFF

# Check MPRJ_IO input/out/en
# 0x10 : Data signal of ps_mprj_in
#        bit 31~0 - ps_mprj_in[31:0] (Read/Write)
# 0x14 : Data signal of ps_mprj_in
#        bit 5~0 - ps_mprj_in[37:32] (Read/Write)
#        others  - reserved
# 0x1c : Data signal of ps_mprj_out
#        bit 31~0 - ps_mprj_out[31:0] (Read)
# 0x20 : Data signal of ps_mprj_out
#        bit 5~0 - ps_mprj_out[37:32] (Read)
#        others  - reserved
# 0x34 : Data signal of ps_mprj_en
#        bit 31~0 - ps_mprj_en[31:0] (Read)
# 0x38 : Data signal of ps_mprj_en
#        bit 5~0 - ps_mprj_en[37:32] (Read)
#        others  - reserved
print ("-> Check MPRJ_IO input/out/en")
print ("   0x10 = ", hex(ipPS.read(0x10)))
print ("   0x14 = ", hex(ipPS.read(0x14)))
print ("   0x1c = ", hex(ipPS.read(0x1c)))
print ("   0x20 = ", hex(ipPS.read(0x20)))
print ("   0x34 = ", hex(ipPS.read(0x34)))
print ("   0x38 = ", hex(ipPS.read(0x38)))

In [ ]:
ol.interrupt_pins # Print out all interrupt pins

In [ ]:
mbIRQ = ol.ps_axil_0.aa_mb_irq # Instance the IRQ from FPGA-side MailBox

In [ ]:
# Create np with 8K/4 (4 bytes per index) size and be initiled to 0
npROM = np.zeros(ROM_SIZE >> 2, dtype=np.uint32)

def load_fw():   
    # ============================================
    # Load firmware (fsic.hex) to memory npROM
    # ============================================
    print ("===========================================")
    print ("-> Load firmware (fsic.hex) to memory npROM")

    npROM_index = 0
    npROM_offset = 0
    fiROM = open("fsic.hex", "r+")

    for line in fiROM:
        # offset header
        if line.startswith('@'):
            # Ignore first char @
            npROM_offset = int(line[1:].strip(b'\x00'.decode()), base = 16)
            npROM_offset = npROM_offset >> 2 # 4byte per offset
            #print (npROM_offset)
            npROM_index = 0
            continue
        #print (line)

        # We suppose the data must be 32bit alignment
        buffer = 0
        bytecount = 0
        for line_byte in line.strip(b'\x00'.decode()).split():
            buffer += int(line_byte, base = 16) << (8 * bytecount)
            bytecount += 1
            # Collect 4 bytes, write to npROM
            if(bytecount == 4):
                npROM[npROM_offset + npROM_index] = buffer
                # Clear buffer and bytecount
                buffer = 0
                bytecount = 0
                npROM_index += 1
                #print (npROM_index)
                continue
        # Fill rest data if not alignment 4 bytes
        if (bytecount != 0):
            npROM[npROM_offset + npROM_index] = buffer
            npROM_index += 1

    fiROM.close()
    print ("-> Finish loading firmware into npROM")
    print ("===========================================")


def cnfg(AxiQspi, clk_phase=0, clk_pol=0):
    print("-> Configure device")
    # Reset the SPI device
    AxiQspi.write(XSP_SRR_OFFSET, XSP_SRR_RESET_MASK)
    # Enable the transmit empty interrupt, which we use to determine progress on the transmission. 
    AxiQspi.write(XSP_IIER_OFFSET, XSP_SR_TX_EMPTY_MASK)
    # Disable the global IPIF interrupt
    AxiQspi.write(XSP_DGIER_OFFSET, 0)
    # Deselect the slave on the SPI bus
    AxiQspi.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)
    # Disable the transmitter, enable Manual Slave Select Assertion, put SPI controller into master mode, and enable it
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg | XSP_CR_MASTER_MODE_MASK | XSP_CR_MANUAL_SS_MASK | XSP_CR_ENABLE_MASK | XSP_CR_TXFIFO_RESET_MASK | XSP_CR_RXFIFO_RESET_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg & ~(XSP_CR_CLK_PHASE_MASK | XSP_CR_CLK_POLARITY_MASK) 
    if clk_phase == 1:
        ControlReg = ControlReg | XSP_CR_CLK_PHASE_MASK
    if clk_pol == 1:
        ControlReg = ControlReg | XSP_CR_CLK_POLARITY_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)

    return 0

def write_tx_fifo(AxiQspi):
    #print("TransferData")
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg & ~XSP_CR_TRANS_INHIBIT_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)

    StatusReg = AxiQspi.read(XSP_SR_OFFSET)
    while (StatusReg & XSP_SR_TX_EMPTY_MASK) == 0:
        StatusReg = AxiQspi.read(XSP_SR_OFFSET)

    #print('XSP_RFO_OFFSET  : 0x{0:08x}'.format(AxiQspi.read(XSP_RFO_OFFSET)))
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg | XSP_CR_TRANS_INHIBIT_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)


def read_rx_fifo(bypass_length, AxiQspi):
    #print("ReadResponse")
    resp = list()
    RxFifoStatus = AxiQspi.read(XSP_SR_OFFSET) & 0x01
    
    # By pass the FIFO data during master issue command and address to slave device
    command_addr_length = bypass_length
    counter = 0    
    
    while RxFifoStatus == 0:
        #temp = AxiQspi.read(XSP_RFO_OFFSET)
        #print('XSP_RFO_OFFSET  : 0x{0:08x}'.format(temp))
        temp = AxiQspi.read(XSP_DRR_OFFSET)
        #print('XSP_DRR_OFFSET  : 0x{0:08x}'.format(temp))    

        counter = counter + 1
        if(counter > command_addr_length):
            resp.append(temp)        
        
        RxFifoStatus = AxiQspi.read(XSP_SR_OFFSET) & 0x01

    return resp

def caravel_start():
    # ============================================
    # Release Reset First before passthrough mode
    # ============================================
    print ("===========================================")
    print ("-> Release Reset First before passthrough mode")
    print ("===========================================")
    # Release Caravel reset
    # 0x10 : Data signal of outpin_ctrl
    #        bit 0  - outpin_ctrl[0] (Read/Write)
    #        others - reserved
    print (ipOUTPIN.read(0x10))
    ipOUTPIN.write(0x10, 1)
    print (ipOUTPIN.read(0x10))
    
def passthru():
    # ============================================
    # Enabling passthrou mode
    # ============================================
    print ("===========================================")
    print ("-> Enabling passthrough mode")
    print ("===========================================")
    cnfg(ip_QSPI)
    # Passthrou mode - Write command
    ip_QSPI.write(XSP_DTR_OFFSET, 0xC4) # Pass-Through (management)
    ip_QSPI.write(XSP_DTR_OFFSET, 0x02) # Command: Write data to memory
    ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Address_byte0
    ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Address_byte1
    ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Address_byte2

    print('-> XSP_TFO_OFFSET  : 0x{0:08x}'.format(ip_QSPI.read(XSP_TFO_OFFSET)))

    ip_QSPI.write(XSP_SSR_OFFSET, 0xFFFFFFFE)
    write_tx_fifo(ip_QSPI)

    print('-> XSP_TFO_OFFSET  : 0x{0:08x}'.format(ip_QSPI.read(XSP_TFO_OFFSET)))
    # ============================================
    # Writing FW into SPIROM
    # ============================================
    # Fill up Tx_FIFO (16) for each write_tx_fifo
    for index in range (ROM_SIZE >> 2):
         # 4 bytes alignment in npROM
        for byte_shift in range(4):
            tmp = int((npROM[index] >> (byte_shift * 8)) & 0xFF)
            ip_QSPI.write(XSP_DTR_OFFSET, tmp) # Write_data
        # TX_FIFO = 16, 4 * 4 = 16
        if((index % 3) == 3):
            write_tx_fifo(ip_QSPI)

    # If rest data is not enough 16 bytes. Tx_FIFO is not empty
        StatusReg = ip_QSPI.read(XSP_SR_OFFSET)
        if ((StatusReg & XSP_SR_TX_EMPTY_MASK) == 0):
             write_tx_fifo(ip_QSPI)
    print ("-> Finish enabling passthrough mode")
    # ============================================
    # Exit passthrou mode, FW will be fetched
    # ============================================
    print ("===========================================")
    print ("-> Exit passthrou mode, FW will be fetched")
    print ("===========================================")
    ip_QSPI.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)
    # Check MPRJ_IO input/out/en
    # 0x10 : Data signal of ps_mprj_in
    #        bit 31~0 - ps_mprj_in[31:0] (Read/Write)
    # 0x14 : Data signal of ps_mprj_in
    #        bit 5~0 - ps_mprj_in[37:32] (Read/Write)
    #        others  - reserved
    # 0x1c : Data signal of ps_mprj_out
    #        bit 31~0 - ps_mprj_out[31:0] (Read)
    # 0x20 : Data signal of ps_mprj_out
    #        bit 5~0 - ps_mprj_out[37:32] (Read)
    #        others  - reserved
    # 0x34 : Data signal of ps_mprj_en
    #        bit 31~0 - ps_mprj_en[31:0] (Read)
    # 0x38 : Data signal of ps_mprj_en
    #        bit 5~0 - ps_mprj_en[37:32] (Read)
    #        others  - reserved
    print ("Check MPRJ_IO input/out/en")
    print ("0x10 = ", hex(ipPS.read(0x10)))
    print ("0x14 = ", hex(ipPS.read(0x14)))
    print ("0x1c = ", hex(ipPS.read(0x1c)))
    print ("0x20 = ", hex(ipPS.read(0x20)))
    print ("0x34 = ", hex(ipPS.read(0x34)))
    print ("0x38 = ", hex(ipPS.read(0x38)))


In [ ]:

# ======================================================= #
# Create asynchronous task for Interrupt Service Routine
# ======================================================= #
async def isr():
    print("-> Waitting for MailBox interrupt")
    while(True):
        await mbIRQ.wait() # Wait for Interrupt
        print("*******************************************")
        print("*******************************************")
        print("-> Interrupt asserted!!!")
        print("*******************************************")
        print("*******************************************")

async def fir_dma_test():
    print("===========================================")
    print("-> Configurate FIR and DMA")
    print("===========================================")
    
    IP_BASE_ADDRESS = 0x60000000
    ADDRESS_RANGE = 0xa000
    mmio = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
    

    # PL_IS Config
    ADDRESS_OFFSET = PL_IS #0x7000
    print("-> mmio.read(PL_IS): ", hex(mmio.read(ADDRESS_OFFSET)))
    mmio.write(ADDRESS_OFFSET, 0x12345671)
    print("-> mmio.read(PL_IS): ", hex(mmio.read(ADDRESS_OFFSET)))
    mmio.write(ADDRESS_OFFSET, 0x12345673)
    print("-> mmio.read(PL_IS): ", hex(mmio.read(ADDRESS_OFFSET)))

    # choose uprj1
    ADDRESS_OFFSET = SOC_CC # 0x5000
    mmio.write(ADDRESS_OFFSET, 0x00000001)
    
    # set len
    ADDRESS_OFFSET = SOC_UP 
    mmio.write(ADDRESS_OFFSET + 0x10, 0x00000040)
    # set coef
    coef = [0x0,0xFFFFFFF6, 0xFFFFFFF7, 0x00000017, 0x00000038, 0x0000003f, 0x00000038, 0x00000017, 0xFFFFFFF7, 0xFFFFFFF6, 0x0]
    ADDRESS_OFFSET = SOC_UP 
    for i in range(11):
        mmio.write(ADDRESS_OFFSET + 0x20 + 4 * i, coef[i])
    
    ##setting mmio for DMA
    ADDRESS_OFFSET = FIR_DMA
    # firDMA_mmio = MMIO(0x60009000, 0x1000)
    mmio.write(ADDRESS_OFFSET + 0x28, 0x00) # s2m disable to clear
    mmio.write(ADDRESS_OFFSET + 0x70, 0x00) # m2s disable to clear
    mmio.write(ADDRESS_OFFSET + 0x20, 0x40) # s2m set buffer len

    firDMA_buf_i = allocate(shape=(1024,), dtype=np.int32)
    firDMA_buf_o = allocate(shape=(1024,), dtype=np.int32)

    mmio.write(ADDRESS_OFFSET + 0x30, firDMA_buf_o.device_address)  # output buffer addr low
    mmio.write(ADDRESS_OFFSET + 0x34, 0x00) # output buffer addr high

    mmio.write(ADDRESS_OFFSET + 0x4C,firDMA_buf_i.device_address)  # input buffer addr low
    mmio.write(ADDRESS_OFFSET + 0x50, 0x00) # input buffer addr high
    mmio.write(ADDRESS_OFFSET + 0x68, 0x40) # m2s set buffer len
    
    for i in range(64):
        firDMA_buf_i[i] = i

    print("-> Start FIR")
    print("-------------------------------------------")
    # fir ap_start
    ADDRESS_OFFSET = SOC_UP 
    mmio.write(ADDRESS_OFFSET, 0x01)

    print("-> Start DMA")
    print("-------------------------------------------")
    # dma_start
    ADDRESS_OFFSET = FIR_DMA
    mmio.write(ADDRESS_OFFSET, 0x01)

    print(f"-> DMA ap_control: {bin(mmio.read(ADDRESS_OFFSET + 0x00))}, FIR ap_control: {bin(mmio.read(SOC_UP + 0x00))}")
    print(f"-> DMA s2m len: {mmio.read(ADDRESS_OFFSET + 0x68)}")
    print(f"-> DMA m2s len: {mmio.read(ADDRESS_OFFSET + 0x20)}")

    print("-------------------------------------------")
    print("-> Wait DMA transfer done")
    while (mmio.read(ADDRESS_OFFSET + 0x10) != 0x01):
        continue
    
    print("-> DMA transfer done!")
    print("-------------------------------------------")

    for i in range(64):
        print(f"result {i:>02d} : {firDMA_buf_o[i]:>5d}")

    ADDRESS_OFFSET = SOC_UP
    print(f"FIR control signal: {bin(mmio.read(0x00))}")
    print ("===========================================")
    print ("ON BOARD VALIDATION FINISH!")
    print ("===========================================")

In [ ]:
async def async_main():
    # Load Firmware into ROM
    load_fw()
    # Start Caravel and Passthrough mode
    caravel_start()
    passthru()
    # Create ISR
    task1 = asyncio.create_task(isr())
    task2 = asyncio.create_task(fir_dma_test())
    await asyncio.sleep(5)
    task1.cancel()
    try:
        await task1
    except asyncio.CancelledError:
        print("-> ISR is cancelled.")

In [ ]:
asyncio.run(async_main())